#*Read Use Case*

In [0]:
%sql
Create catalog if not exists telecom_catalog_assign;

create schema if not exists telecom_catalog_assign.landing_zone;

create volume if not exists telecom_catalog_assign.landing_zone.landing_vol

In [0]:
dbutils.fs.mkdirs('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/')

dbutils.fs.mkdirs('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/')

dbutils.fs.mkdirs('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/')                  

In [0]:
dbutils.fs.put('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv', '101,Arun,31,Chennai,PREPAID\n102,Meera,45,Bangalore,POSTPAID\n103,Irfan,29,Hyderabad,PREPAID\n104,Raj,52,Mumbai,POSTPAID\n105,27,Delhi,PREPAID\n106,Sneha,abc,Pune,PREPAID',overwrite=True)

dbutils.fs.put('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage_tsv.txt','customer_id\tvoice_mins\tdata_mb\tsms_count\n101\t320\t1500\t20\n102\t120\t4000\t5\n103\t540\t600\t52\n104\t45\t200\t2\n105\t0\t0\t0',overwrite=True)

dbutils.fs.put('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower_logs_region1.txt','event_id|customer_id|tower_id|signal_strength|timestamp\n5001|101|TWR01|-80|2025-01-10 10:21:54\n5004|104|TWR05|-75|2025-01-10 11:01:12',overwrite=True)



In [0]:
dbutils.fs.mv('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv','/Volumes/telecom_catalog_assign/landing_zone/landing_vol/Customer/customer1.csv')

In [0]:
dbutils.fs.mv('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage_tsv.txt','/Volumes/telecom_catalog_assign/landing_zone/landing_vol/Usage/usage_tsv.txt')

In [0]:

dbutils.fs.mv('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower_logs_region1.txt','/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower_logs_region2.txt')

In [0]:
print(dbutils.fs.ls('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/Usage/usage_tsv.txt'))
print(dbutils.fs.ls('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/Customer/customer.csv'))
print(dbutils.fs.ls('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower_logs_region2.txt'))

In [0]:
dbutils.fs.cp('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/sub/tower_logs_region5.txt', '/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/sub/tower_logs_region8.csv')


In [0]:
spark.read.options(pathglobfilter="*.csv",recursiveFileLookup="true", header="True",sep='|',inferSchema="true").csv('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower').display()


In [0]:
df=spark.read.option('header', True).option('sep', '|').option('inferSchema', True).format('csv').load(['/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower_logs_region2.txt','/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/sub/tower_logs_region5.txt']).display()


In [0]:
spark.read.option('header', True).option('inferSchema', True).format('csv').load('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/Customer/customer.csv').display()

spark.read.option('header',False).option('inferschema',False).csv('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/Customer/customer.csv').display()

spark.read.options(header=True, inferSchema=True).csv('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/Customer/customer.csv').display()

spark.read.options(header=False, inferSchema=False).csv('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/Customer/customer.csv').display()

#Write a note on What changed when we use header or inferSchema with true/false?

With header true spark with consider the first row as header fro those columns and with header false spark will assign the column name as c1,c2,..cn

With inferschema true spark will guess the column data type and with inferschema false spark will assign the column data type as string

##*How schema inference handled “abc” in age?*

It will consider as string



In [0]:
%python
df=spark.read.options(header=True, inferschema=True).csv('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/Customer/customer.csv').toDF('Id','Name','Age','City','Plan').display()


In [0]:
dbutils.fs.cp('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage_tsv.txt','/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage_tsv.csv')

In [0]:
df_schema='id int,name string,age int,city string,plan string'
df=spark.read.schema(df_schema).csv('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/Customer/customer.csv').display()

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

df_schema=StructType([
  StructField("customer_id", IntegerType(), True),
  StructField('voice_mins', IntegerType(), True),
  StructField('data_mb', IntegerType(), True),
  StructField('sms_count', IntegerType(), True)])

df=spark.read.schema(df_schema).csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage_tsv.csv").display()

#*Write Use Case*

In [0]:

#instantiation
from pyspark.sql.session from SparkSession
spark= SparkSession.builder.getorCreate()

In [0]:
df_data=spark.read.csv('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/Customer/customer.csv',header=False,inferSchema=True,sep=',').toDF('Id','Name','Age','City','Plan')

df_data.write.csv('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/Customer/Write_Customer.csv',header=True,sep=',',mode='overwrite')


In [0]:
df_write=spark.read.csv('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/Usage/usage.CSV',header=True,sep=',')

df_write.write.csv('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage_usecase.CSV',header=True,sep=',',mode='append')

In [0]:
df_write=spark.read.csv('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower_logs_region1.txt',header=True, sep='|')

display(df_write)

df_write.write.csv('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower_logs_region1.csv',header=True, sep=',',mode='overwrite')

In [0]:
spark.read.csv('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower_logs_region1.csv/part-00000-tid-6608864050204356626-751165ae-9781-40b8-b255-55a81f6ed95f-238-1-c000.csv',header=True).display()

#####Write Operations (Data Conversion/Schema migration)– JSON Format Usecases

In [0]:
df_wr=spark.read.csv('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/Customer/customer.csv',header=True,sep=',').toDF('Id','Name','Age','City','Plan')

df_wr.write.json('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/Customer/Customer_json',mode='overwrite')

In [0]:
df_wr=spark.read.csv('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/Usage/usage.CSV',header=True,sep=',')

df_wr.write.json('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/Usage/usage_json',mode='append',compression='snappy')

In [0]:
df_wr=spark.read.csv('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower_logs_region1.txt',header=True,sep='|')
df_wr.write.json('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower_json',mode='ignore')

display(df_wr.limit(5))

####Write Operations (Data Conversion/Schema migration) – Parquet Format Usecases

In [0]:
df_write=spark.read.csv('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/Customer/customer.csv',header=True, inferSchema=True,sep=',')

df_write.write.parquet('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/Customer_parguet2',mode='overwrite',compression='gzip')


In [0]:
df_wr=spark.read.csv('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/Usage/usage.CSV',header=True,inferSchema=True,sep=',')

df_wr.write.parquet('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/Usage/parquet_usage2',mode='error')

In [0]:
df_wr=spark.read.csv('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower_logs_region1.txt',header=True,inferSchema=True,sep='|')

df_wr.write.parquet('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/parquet_tower',mode='overwrite',compression='gzip')

display(df_wr.limit(5))

####Write Operations (Data Conversion/Schema migration) – Orc Format Usecases

In [0]:
df=spark.read.csv('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/Customer/customer.csv',header=True,inferSchema=True,sep=',')

df.write.orc('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/Customer/orc_customer',mode='overwrite')

In [0]:
df=spark.read.csv('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/Usage/usage.CSV',header=True,inferSchema=True,sep=',')

df.write.orc('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/Usage/orc_usage',mode='append')

In [0]:
df=spark.read.csv('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower_logs_region1.txt',header=True,inferSchema=True,sep='|')

df.write.orc('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/orc_tower',mode='overwrite')

display(df.limit(5))

####Write Operations (Data Conversion/Schema migration) – Delta Format Usecases

In [0]:
df=spark.read.csv('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/Customer/customer.csv',header=True,inferSchema=True,sep=',')

df.write.format('delta').save('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/Customer/delta_customer',mode='overwrite')

In [0]:
df=spark.read.csv('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/Usage/usage.CSV',header=True,inferSchema=True,sep=',')

df.write.format('delta').save('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/Usage/delta_usage',mode='append')

In [0]:
df=spark.read.csv('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower_logs_region1.txt',header=True,inferSchema=True,sep='|')
df.write.format('delta').save('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/delta_tower',mode='overwrite')

display(df.limit(5))

####Write Operations (Lakehouse Usecases) – Delta table Usecases

In [0]:
df=spark.read.csv('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/Customer/customer.csv',header=True,inferSchema=True,sep=',')
df.write.format('delta').saveAsTable('telecom_catalog_assign.landing_zone.customer')

In [0]:
df=spark.read.csv('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/Usage/usage.CSV',header=True,inferSchema=True,sep=',')
df.write.format('delta').saveAsTable('telecom_catalog_assign.landing_zone.usage',mode='overwrite')

In [0]:
%sql
Drop table telecom_catalog_assign.landing_zone.customer;

In [0]:
spark.sql('select * from telecom_catalog_assign.landing_zone.usage').display()

####Write customer data using insertInto() in a new table and find the behavior

In [0]:
df_customer=spark.read.csv('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/Customer/customer.csv',header=False,inferSchema=False).toDF('id','name','age','city','plan')

In [0]:
%sql
create table if not exists telecom_catalog_assign.landing_zone.customer
(
  id string,
  name string,
  age string,
  city string,
  plan string
)

In [0]:
df_customer.write.insertInto('telecom_catalog_assign.landing_zone.Customer')

In [0]:
%sql
select * from telecom_catalog_assign.landing_zone.Customer;

####Write usage data using insertTable() with overwrite mode

In [0]:
df_usage=spark.read.csv('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/Usage/usage.CSV',header=True,inferSchema=True,sep=',')

In [0]:
%sql
create table telecom_catalog_assign.landing_zone.usage_tab
(
customer_id int,
voice_mins int,
data_mb int,
sms_count int
);

In [0]:
df_usage.write.insertInto('telecom_catalog_assign.landing_zone.usage_tab')

In [0]:
%sql
select * from telecom_catalog_assign.landing_zone.usage_tab;

####Write customer data into XML format using rowTag as cust

In [0]:
df_customer_xml=spark.read.csv('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/Customer/customer.csv',header=False,inferSchema=True,sep=',').toDF('id','name','age','city','plan')

df_customer_xml.write.format('xml').mode('overwrite').save('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/Customer/customer_xml',rowtag='customer')

display(df_customer_xml)

####Write usage data into XML format using overwrite mode with the rowTag as usage

In [0]:
df_usage_xml=spark.read.csv('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/Usage/usage.CSV',header=True,inferSchema=True,sep=',')

df_usage_xml.write.format('xml').mode('overwrite').save('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/Usage/usage_xml',rowtag='usage')

display(df_usage_xml)

#####Try to do permutation and combination of performing Schema Migration & Data Conversion operations like...
Read any one of the above orc data in a dataframe and write it to dbfs in a parquet format

Read any one of the above parquet data in a dataframe and write it to dbfs in a delta format

Read any one of the above delta data in a dataframe and write it to dbfs in a xml format

Read any one of the above delta table in a dataframe and write it to dbfs in a json format

Read any one of the above delta table in a dataframe and write it to another table

#
Read any one of the above orc data in a dataframe and write it to dbfs in a parquet format

In [0]:
df_orc_cust=spark.read.orc('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/Customer/orc_customer/part-00000-tid-4614474085161580138-1def542b-e003-4633-8f93-bb75add2e33c-265-1-c000.snappy.orc')

df_orc_cust.write.parquet('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/Customer/cust_orc_parquet2')

Read any one of the above parquet data in a dataframe and write it to dbfs in a delta format


In [0]:
df_cust_parque_delta=spark.read.parquet('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/Usage/parquet_usage/part-00000-tid-9057186356024158004-a0a4facb-5201-4c44-b803-f0eaa2b97f8c-239-1.c000.snappy.parquet')

df_cust_parque_delta.write.format('delta').save('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/Usage/usage_parq_delta',overwrite=True)

Read any one of the above delta data in a dataframe and write it to dbfs in a xml format

In [0]:
df_delta_xml=spark.read.format('delta').load('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/delta_tower')

df_delta_xml.write.format('xml').mode('overwrite').option('rowTag','tower').save('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower_delta_xml')

Read any one of the above delta table in a dataframe and write it to dbfs in a json format

In [0]:
df_delta_json=spark.read.table('telecom_catalog_assign.landing_zone.customer')

df_delta_json.write.format('json').mode('overwrite').save('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/Customer/cust_delta_json')

Read any one of the above delta table in a dataframe and write it to another table

In [0]:
df_delta_delta=spark.read.table('telecom_catalog_assign.landing_zone.usage')

df_delta_delta.write.format('delta').saveAsTable('telecom_catalog_assign.landing_zone.usage2')